In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

In [5]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),    
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        73792     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          1

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).batch(64) # 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [9]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_acc',
    save_best_only=True)

In [10]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [11]:
history = model.fit(train_data,
                    epochs=40,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 15:50:53.054154: {'loss': 1.7689715034484863, 'accuracy': 0.413275, 'val_loss': 1.6467225285852032, 'val_accuracy': 0.4023}


Epoch 1 ends at 15:51:12.920479: {'loss': 1.1931104391098022, 'accuracy': 0.5794, 'val_loss': 1.6188946605487993, 'val_accuracy': 0.4721}


Epoch 2 ends at 15:51:33.312337: {'loss': 1.022985613155365, 'accuracy': 0.639775, 'val_loss': 1.0590684026669546, 'val_accuracy': 0.6244}


Epoch 3 ends at 15:51:54.570498: {'loss': 0.9311705169677734, 'accuracy': 0.67215, 'val_loss': 1.0052817870097555, 'val_accuracy': 0.6538}


Epoch 4 ends at 15:52:16.664424: {'loss': 0.8754433826446533, 'accuracy': 0.6957, 'val_loss': 1.0436692146738624, 'val_accuracy': 0.6357}


...

Epoch 35 ends at 16:06:28.514840: {'loss': 0.4231377313375473, 'accuracy': 0.847825, 'val_loss': 0.7607798874378204, 'val_accuracy': 0.7611}


Epoch 36 ends at 16:06:59.451125: {'loss': 0.41387437760829926, 'accuracy': 0.85155, 'val_loss': 0.8068094667355725, 'val_accuracy': 0.7442}


Epoch 37 ends at 16:07:32.179621: {'loss': 0.40672331132888795, 'accuracy': 0.853875, 'val_loss': 0.7775436863777744, 'val_accuracy': 0.7512}


Epoch 38 ends at 16:08:04.944020: {'loss': 0.400910808634758, 'accuracy': 0.856, 'val_loss': 0.7520571986960757, 'val_accuracy': 0.7606}


Epoch 39 ends at 16:08:40.524224: {'loss': 0.39829282822608947, 'accuracy': 0.858925, 'val_loss': 0.730420480678036, 'val_accuracy': 0.7662}


In [12]:
model.evaluate(test_data)

157/157 [==============================].7717 - accuracy: 0.76 - 0s 64ms/step - loss: 0.7664 - accuracy: 0.76 - 0s 57ms/step - loss: 0.7502 - accuracy: 0.78 - 0s 55ms/step - loss: 0.7453 - accuracy: 0.76 - 0s 57ms/step - loss: 0.7033 - accuracy: 0.78 - 0s 54ms/step - loss: 0.6801 - accuracy: 0.78 - 0s 53ms/step - loss: 0.6759 - accuracy: 0.78 - 0s 51ms/step - loss: 0.7019 - accuracy: 0.78 - 0s 49ms/step - loss: 0.6965 - accuracy: 0.77 - 0s 49ms/step - loss: 0.6960 - accuracy: 0.77 - 1s 48ms/step - loss: 0.6779 - accuracy: 0.77 - 1s 49ms/step - loss: 0.6871 - accuracy: 0.77 - 1s 48ms/step - loss: 0.6804 - accuracy: 0.77 - 1s 47ms/step - loss: 0.6905 - accuracy: 0.77 - 1s 47ms/step - loss: 0.6940 - accuracy: 0.77 - 1s 47ms/step - loss: 0.7208 - accuracy: 0.76 - 1s 47ms/step - loss: 0.7116 - accuracy: 0.76 - 1s 46ms/step - loss: 0.7108 - accuracy: 0.77 - 1s 45ms/step - loss: 0.7088 - accuracy: 0.77 - 1s 45ms/step - loss: 0.7127 - accuracy: 0.76 - 1s 45ms/step - loss: 0.7021 - accuracy: 0.

[0.7439867550400412, 0.7598]

In [13]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))

OSError: SavedModel file does not exist at: checkpoints/20191028-155031.hdf5/{saved_model.pbtxt|saved_model.pb}